In [ ]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(dplyr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)

source("src/database/query/data.R")
source("src/database/startup.R")
source("src/load/tools.R")
source("src/load/ITA.R")
source("notebooks/ds_regionali/procedure/common_steps.R")

conns <- load_dbs()

In [ ]:
load.italian_boundaries("state")

In [ ]:
regional_datasets <- tribble(
    ~reference_state, ~original_dataset, ~merged_dataset,
    "Piemonte", "ARPAPiemonte", "PIE",
    "Liguria", "ARPAL", "LIG",
    "Lombardia", "ARPALombardia", "LOM",
    "Veneto", "ARPAV", "VEN",
    "Trentino-Alto Adige", "TAA", "TAA2",
    "Friuli-Venezia Giulia", "ARPAFVG", "FVG",
    "Emilia-Romagna", "ARPAE", "ER",
    "Toscana", "SIRToscana", "TOS",
    "Umbria", "ARPAUmbria", "UMB",
    "Marche", "ARPAM", "MAR",
    "Valle D'Aosta", "SCIA", "VDA"
)
reg_boundaries <- load.italian_boundaries("state") |> filter(shapeName %in% c(regional_datasets$reference_state, "Friuli Venezia Giulia"))


In [ ]:
data <- query_checkpoint_data(regional_datasets$merged_dataset, "merged", conns$data)
meta <- query_checkpoint_meta(regional_datasets$merged_dataset, "merged", conns$meta)

In [ ]:
pd <- ymonthly_availabilities(data |> filter(year(date) >= 1990L), FALSE)
ggsave("plots/state_availabilities.pdf", pd$plot + facet_wrap(~ dataset, ncol = 3L), width = 12)
pd$plot + facet_wrap(~ dataset, ncol = 3L)

In [ ]:
ps <- spatial_availabilities(pd$data, meta, reg_boundaries, n_years_threshold = 5L)
ggsave("plots/spatial_availability.pdf", ps$plot)
ps$plot

In [ ]:
p <- plot_available_by_elevation(ps$data, meta |> collect()) + facet_wrap(~ dataset, ncol = 3L)
ggsave("plots/availability_by_elevation.pdf", p, width = 12, height = 8, units = "in", dpi = 300)
p

In [ ]:
p <- plot_available_by_elevation(ps$data, meta |> collect())
p

In [ ]:
scia_synop_mareo_meta <- query_checkpoint_meta("SCIA", "qc1", conns$data) |> filter(network %in% c("Sinottica", "Mareografica"), lat > 42, !(state %in% c("Abruzzo", "Lazio")))
scia_synop_mareo_data <- query_checkpoint_data("SCIA", "qc1", conns$data) |> semi_join(scia_synop_mareo_meta, by = "sensor_key")
pd <- ymonthly_availabilities(scia_synop_mareo_data |> filter(year(date) >= 1990L, valid), FALSE)
pd$plot

In [ ]:
ps <- spatial_availabilities(pd$data, scia_synop_mareo_meta, reg_boundaries, n_years_threshold = 5L)
ps$plot